In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd

nav = webdriver.Chrome()

# importar/visualizar a base de dados
tabela_produtos = pd.read_excel('buscas.xlsx')
display(tabela_produtos)


,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64 gb,mini watch,3000,3500
1,rtx 3060,zota galax,4000,4500


### Definição das Funçoes de busca no google e no buscape

In [2]:
import time
def busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo):

     # entrar no google
    nav.get('https://www.google.com.br/')

    #tratar os valores que vieram da tabela
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()

    lista_termos_banidos = termos_banidos.split(' ')
    lista_termos_produtos = produto.split(' ')
    preco_maximo = float(preco_maximo)
    preco_minimo = float(preco_minimo)

    #Pesquisar o nome do produto no google
    nav.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(produto)
    nav.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

    # clicar na aba shopping
    elementos = nav.find_elements(By.CLASS_NAME,'hdtb-mitem')
    for item in elementos:
        if 'Shopping' in item.text:
            item.click()
            break

    #pegar a lista de resultados da busca no google shopping
    lista_resultados = nav.find_elements(By.CLASS_NAME, 'sh-dgr__grid-result')     

    #Para cada resultado ele vai verificar se o resultado corresponde a todas as nossas condições
    
    lista_ofertas = [] #Lista que a funçao vai me dar como resposta
    for resultado in lista_resultados:
        
        nome = resultado.find_element(By.CLASS_NAME, 'Xjkr3b').text
        nome = nome.lower()

        #verificação do nome, se no nome tem algum termo banido
        tem_termos_banidos = False
        
        for palavra in lista_termos_banidos:
            if palavra in nome:
                tem_termos_banidos = True

        #Verificar se no nome tem todos os termos do produto
        tem_todos_termos_produto = True
        for palavra in lista_termos_produtos:
            if palavra not in nome:
                tem_todos_termos_produto = False
    
        if not tem_termos_banidos and tem_todos_termos_produto: # Verificando o nome.
            
            preco = resultado.find_element(By.CLASS_NAME, 'a8Pemb').text
            preco = preco.replace('R$','').replace(' ','').replace('.','').replace(',','.')
            preco = float(preco)
        
            #Verificando se o preço esta dentro do minimo e do máximo.
            
            if preco_minimo <= preco <= preco_maximo:
                #se tem_termos_banidos - False e o tem_termos_produto - True
                elemento_link = resultado.find_element(By.CLASS_NAME, 'aULzUe')
                elemento_pai = elemento_link.find_element(By.XPATH, '..')
                link = elemento_pai.get_attribute('href')
                lista_ofertas.append((link, preco, nome))
        
    return lista_ofertas
            
    
def busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo):
    # Tratar os valores da função
    preco_maximo = float(preco_maximo)
    preco_minimo = float(preco_minimo)
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(' ')
    lista_termos_produtos = produto.split(' ')


    # Pesquisar pelo produto no buscape
    nav.get('https://www.buscape.com.br/')

    nav.find_element(By.CLASS_NAME, 'search-bar__text-box').send_keys(produto, Keys.ENTER)
    

    # Pegar a lista de resultados na busca do buscape
    time.sleep(5)
    lista_resultado = nav.find_elements(By.CLASS_NAME, 'Cell_Content__1630r')

    #Ir para cada resultado.
    
    lista_ofertas = []
    for resultado in lista_resultado:
        try:

            preco = resultado.find_element(By.CLASS_NAME, 'CellPrice_MainValue__3s0iP').text
            nome = resultado.get_attribute('title')
            nome = nome.lower()
            link = resultado.get_attribute('href')

            #verificação do nome, se no nome tem algum termo banido
            tem_termos_banidos = False
            
            for palavra in lista_termos_banidos:
                if palavra in nome:
                    tem_termos_banidos = True

            #Verificar se no nome tem todos os termos do produto
            tem_todos_termos_produto = True
            for palavra in lista_termos_produtos:
                if palavra not in nome:
                    tem_todos_termos_produto = False
            
            if not tem_termos_banidos and tem_todos_termos_produto:
                preco = preco.replace('R$','').replace(' ','').replace('.','').replace(',','.')
                preco = float(preco)
                if preco_minimo <= preco <= preco_maximo:
                    lista_ofertas.append((nome,preco,link)) #Colocamos dois parênteses porque o append só aceite mais de um valor dentro de uma tupla.           
        except:
            pass
    
    return lista_ofertas

#produto = 'iphone 12 64 gb'
#termos_banidos = 'mini watch'
#preco_minimo = 3000
#preco_maximo = 3500

#lista_ofertas_google_shopping = busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo)
#print(lista_ofertas_google_shopping)

#lista_ofertas_buscape = busca_buscape(nav, produto, termos_banidos,preco_minimo,preco_maximo)





### Construção da nossa lista de ofertas encontrada.

In [3]:
tabela_ofertas = pd.DataFrame()

for linha in tabela_produtos.index:
    produto = tabela_produtos.loc[linha, "Nome"]
    termos_banidos = tabela_produtos.loc[linha, "Termos banidos"]
    preco_minimo = tabela_produtos.loc[linha, "Preço mínimo"]
    preco_maximo = tabela_produtos.loc[linha, "Preço máximo"]
    
    lista_ofertas_google_shopping = busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_google_shopping:
        tabela_google_shopping = pd.DataFrame(lista_ofertas_google_shopping, columns=['produto', 'preco', 'link'])
        tabela_ofertas = tabela_ofertas.append(tabela_google_shopping)
    else:
        tabela_google_shopping = None # None quer dizer que se não encontrar nada , retorna uma tabela vazia.
        
    lista_ofertas_buscape = busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_buscape:
        tabela_buscape = pd.DataFrame(lista_ofertas_buscape, columns=['produto', 'preco', 'link'])
        tabela_ofertas = tabela_ofertas.append(tabela_buscape)
    else:
        tabela_buscape = None # None quer dizer que se não encontrar nada , retorna uma tabela vazia.
        

display(tabela_ofertas)


,produto,preco,link
0,https://www.google.com.br/url?url=https://www....,3136.42,"iphone se branco, com tela de 4,7, 4g, 64 gb e..."
0,smartphone apple iphone xr 64gb ios 12.0 mp,3428.98,https://www.buscape.com.br/celular/smartphone-...
0,https://www.google.com.br/url?url=https://www....,4499.00,rtx 3060 leadtek lhr 12gb branca
1,https://www.google.com.br/url?url=https://www....,4499.90,placa de vídeo geforce rtx 3060 v2 dual 12gb g...
2,https://www.google.com.br/url?url=https://www....,4199.00,placa de vídeo asus tuf rtx 3060 o12g v2 gamin...
3,https://www.google.com.br/url?url=https://s.cl...,4234.87,asus rog strix nvidia geforce rtx 3060 oc edit...
4,https://www.google.com.br/url?url=https://s.cl...,4431.03,colorido igame geforce rtx 3060 ultra w placa ...
5,https://www.google.com.br/url?url=https://s.cl...,4309.51,ats rtx3060 o12g placa gráfica de jogos gpu pl...
6,https://www.google.com.br/url?url=https://www....,4253.99,placa de video rtx 3060 ti aorus elite lhr 8gb...
7,https://www.google.com.br/url?url=https://www....,4500.00,placa de vídeo palit geforce rtx 3060 stormx l...


### Exportar a base de ofertas para o Excel

In [4]:
tabela_ofertas = tabela_ofertas.reset_index(drop=True) # Isso é para resetar o indice da minha tabela de Ofertas.
tabela_ofertas.to_excel("Ofertas.xlsx", index=False) # Remove a coluna de indice

### Enviar e-mail

In [5]:
# enviar por e-mail o resultado da tabela
import win32com.client as win32

#verificando se existe alguma oferta dentro da tabela de ofertas
if len(tabela_ofertas.index) > 0:
    # vou enviar email
    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = 'rosicreferreira@gmail.com'
    mail.Subject = 'Produto(s) Encontrado(s) na faixa de preço desejada'
    mail.HTMLBody = f"""
    <p>Prezados,</p>
    <p>Encontramos alguns produtos em oferta dentro da faixa de preço desejada. Segue tabela com detalhes</p>
    {tabela_ofertas.to_html(index=False)}
    <p>Qualquer dúvida estou à disposição</p>
    <p>Att.,</p>
    """
    
    mail.Send()

nav.quit()  
